In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("drive/MyDrive/Bootcamp_Henry/pro_final")
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/Bootcamp_Henry/pro_final


# **Objetivo del Cuaderno Jupyter**
- A partir de los archivos recibidos del exterior que contienen los datos de la plataforma `YELP` y apoyandonos en el `Maestro de Categorías` realizado en el equipo, realizar el ETL local de las tablas de yelp que se muestran en la imagen.
<img src="0_TABLAS_YELP.png">


Importando Librerias Necesarias

In [ ]:
import time
import pandas as pd
import datetime
import requests
import gc

## Importando el archivo `business.pkl` tal como viene del exterior

In [ ]:
business = pd.read_pickle('datasets/yelp/archivos_originales/business.pkl')

In [ ]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Por alguna Razón cargo 28 Columnas.. Las 14 Con datos reales y 14 con datos no reales

In [ ]:
business = business.iloc[:,0:14]

In [ ]:
for column in business.columns:
  print(f'Columna --> {column} Cantidad de Nulos --> {business[column].isnull().sum()}')

Columna --> business_id Cantidad de Nulos --> 0
Columna --> name Cantidad de Nulos --> 0
Columna --> address Cantidad de Nulos --> 0
Columna --> city Cantidad de Nulos --> 0
Columna --> state Cantidad de Nulos --> 3
Columna --> postal_code Cantidad de Nulos --> 0
Columna --> latitude Cantidad de Nulos --> 0
Columna --> longitude Cantidad de Nulos --> 0
Columna --> stars Cantidad de Nulos --> 0
Columna --> review_count Cantidad de Nulos --> 0
Columna --> is_open Cantidad de Nulos --> 0
Columna --> attributes Cantidad de Nulos --> 13744
Columna --> categories Cantidad de Nulos --> 103
Columna --> hours Cantidad de Nulos --> 23223


En la visualizacion anterior (cuando se hizo el head())se ve como hay un Registro con city = "Philadelphia" y state = "CA" lo cual no es correcto.  Además de por estudio anterior sabemos que hay Negocios que no Pertenecen a Estados Unidos, además de los 3 registros que traen NAN

Obteniendo codigos postales

In [ ]:
lista_codigos_postales = business['postal_code'].unique().tolist()

print(f'En total hay {len(lista_codigos_postales)} Codigos Postales distintos')

En total hay 3362 Codigos Postales distintos


Hemos visto en sondeos que las ciudades si corresponden siempre con codigos postales, pero que hay veces no corresponden con los estados. Ulilizaremos la Api Gratuita de Codigos Postales de Estados Unidos para obtener los Estados que corresponden a cada Codigo Postal. Ya con esa informacion corregiremos los estados o `state` de todo el dataframe

In [ ]:
estados_obtenidos_api = []
for i in range(len(lista_codigos_postales)):
  codigo_postal = lista_codigos_postales[i]
  diccionario = {}
  try:
    codigo_numero = int(lista_codigos_postales[i])
    url = f"http://api.zippopotam.us/us/{codigo_numero}"
    response = requests.get(url)
    if response.status_code == 200:
      data = response.json()
      estado = data['places'][0]['state abbreviation']
      diccionario['codigo_postal'] = codigo_postal
      diccionario['state'] = estado
    else:
      diccionario['codigo_postal'] = codigo_postal
      diccionario['state'] = 'BORRAR REGISTRO'
  except:
    diccionario['codigo_postal'] = codigo_postal
    diccionario['state'] = 'BORRAR REGISTRO'

  estados_obtenidos_api.append(diccionario)
  time.sleep(2)

codigos_postales_api = pd.DataFrame(estados_obtenidos_api)


In [ ]:
codigos_postales_api['state'].unique().tolist()

['BORRAR REGISTRO',
 'LA',
 'IL',
 'IN',
 'TN',
 'NC',
 'ID',
 'PA',
 'FL',
 'CA',
 'MO',
 'AZ',
 'NV',
 'GA',
 'OH',
 'WA',
 'MT',
 'CO',
 'DE',
 'UT',
 'NM',
 'TX',
 'MI',
 'MD',
 'MN',
 'NY']

Bien ahora debemos interactuar con el dataframe `business` para hacer la correcta asignación de estados... Los registros que queden marcados con state = "BORRAR REGISTRO" se tratan de ubicaciones que no pertenecen a Estados Unidos. Todos los que sondeamos manualmente pertenecían a Canada

Recordando como viene business

In [ ]:
business.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None


In [ ]:
"""
Columnas en business
colomna 4  --> state
columna 5  --> postal_code
"""


for i in range(business.shape[0]):
  codigo_postal = business.iloc[i,5]
  # Filtrando el dataframe que se obtuvo de la Api de Codigos Postales
  mask = codigos_postales_api['codigo_postal'] == codigo_postal
  business.iloc[i,4] = codigos_postales_api.loc[mask]['state'].values[0]

business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


Se deben Borrar los registros que no pertenecen a Estados Unidos

In [ ]:
mask = business['state'] != 'BORRAR REGISTRO'
business = business.loc[mask]

Vamos a filtrar en `business` solo aquellos negocios que sean restaurantes

### Importacion del archivo que en el EDR se llama `y_categories_business` como quedó despues de haber hecho el Maestro de Categorias

In [ ]:
categorias_yelp = pd.read_parquet('datasets/categorias/yelp_categories_final.parquet')

In [ ]:
print(categorias_yelp.shape)
categorias_yelp.head()

(668662, 5)


,business_id,category_id_yelp,category,category_general,id_category_general
0,Pns2l4eNsfO8kk83dixA6A,1,doctors,Salud y Cuidado,6
1,Pns2l4eNsfO8kk83dixA6A,2,traditional chinese medicine,Salud y Cuidado,6
2,Pns2l4eNsfO8kk83dixA6A,3,naturopathic/holistic,Salud y Cuidado,6
3,Pns2l4eNsfO8kk83dixA6A,4,acupuncture,Salud y Cuidado,6
4,Pns2l4eNsfO8kk83dixA6A,5,health & medical,Salud y Cuidado,6


## Filtrado

In [ ]:
# Filtro en Categorias por alimentos y bebidas y obtengo una lista de los negocios
mask = categorias_yelp['category_general'] == 'Alimentos y Bebidas'
lista_business_alimentos_bebidas = categorias_yelp.loc[mask]['business_id'].unique().tolist()

# ahora con la lista de los business_id se filtra a la tabla business
mask = business['business_id'].isin(lista_business_alimentos_bebidas)
business = business.loc[mask]

print(f'Quedaron {business.shape[0]} negocios para el estudio')

Quedaron 28470 negocios para el estudio


Veamos como sigue la situacion de Nulos

In [ ]:
for column in business.columns:
  print(f'Columna --> {column} Cantidad de Nulos --> {business[column].isnull().sum()}')

Columna --> business_id Cantidad de Nulos --> 0
Columna --> name Cantidad de Nulos --> 0
Columna --> address Cantidad de Nulos --> 0
Columna --> city Cantidad de Nulos --> 0
Columna --> state Cantidad de Nulos --> 0
Columna --> postal_code Cantidad de Nulos --> 0
Columna --> latitude Cantidad de Nulos --> 0
Columna --> longitude Cantidad de Nulos --> 0
Columna --> stars Cantidad de Nulos --> 0
Columna --> review_count Cantidad de Nulos --> 0
Columna --> is_open Cantidad de Nulos --> 0
Columna --> attributes Cantidad de Nulos --> 1894
Columna --> categories Cantidad de Nulos --> 0
Columna --> hours Cantidad de Nulos --> 2361


Dado que quedaron pocos datos al filtrar permaneceré con todos los registros. Lo único es que vamos a cambiarlos por "No Disponible" para identificar y evitar posibles errores con Nulos

In [ ]:
business.loc[:, 'attributes'] = business['attributes'].fillna('No Disponible')

business.loc[:, 'hours'] = business['hours'].fillna('No Disponible')

La Columna `categories` ya no debe estar acá ya que fue extraida por el equipo que realizó el maestro de Categorias

In [ ]:
business = business.drop(columns=['categories'])

Necesitamos extraer el `business_id` junto con `attributes` para armar la tabla `y_attributes_business`

In [ ]:
attributes = business[['business_id', 'attributes']]
attributes.head()

,business_id,attributes
1,mpf3x-BjTdTEA3yCZrAYPw,{'BusinessAcceptsCreditCards': 'True'}
2,tUFrWirKiKi_TAnsVWINQQ,"{'BikeParking': 'True', 'BusinessAcceptsCredit..."
10,UJsufbvfyfONHeWdvAHKjA,"{'RestaurantsPriceRange2': '2', 'BikeParking':..."
37,mGFsPRHdE6SMV1gYQ6GzgQ,"{'BusinessParking': '{'garage': False, 'street..."
44,mglME3E7esPSpIF5bias8w,"{'RestaurantsPriceRange2': '3', 'BikeParking':..."


Elimino los registros `No Disponible` de esta tabla `attributes`

In [ ]:
attributes = attributes[attributes['attributes'] != 'No Disponible']
attributes.head()

,business_id,attributes
1,mpf3x-BjTdTEA3yCZrAYPw,{'BusinessAcceptsCreditCards': 'True'}
2,tUFrWirKiKi_TAnsVWINQQ,"{'BikeParking': 'True', 'BusinessAcceptsCredit..."
10,UJsufbvfyfONHeWdvAHKjA,"{'RestaurantsPriceRange2': '2', 'BikeParking':..."
37,mGFsPRHdE6SMV1gYQ6GzgQ,"{'BusinessParking': '{'garage': False, 'street..."
44,mglME3E7esPSpIF5bias8w,"{'RestaurantsPriceRange2': '3', 'BikeParking':..."


Seguiré trabajando con la tabla `y_business`. Para ello debo borrar la columna `attribute`

In [ ]:
business = business.drop(columns=['attributes'])
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,hours
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
10,UJsufbvfyfONHeWdvAHKjA,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,FL,34639,28.190459,-82.45738,3.5,6,1,"{'Monday': '9:30-21:30', 'Tuesday': '9:30-21:3..."
37,mGFsPRHdE6SMV1gYQ6GzgQ,T-Mobile,4735 Concord Pike,Wilmington,DE,19803,39.822176,-75.546007,2.0,6,1,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-20:0', ..."
44,mglME3E7esPSpIF5bias8w,Pier 1,1014 Baltimore Pike,Springfield,PA,19064,39.91527,-75.342681,2.5,8,0,"{'Monday': '10:0-20:0', 'Tuesday': '10:0-20:0'..."


Debo crear una funcion que retorne los Horarios de los distintos dias

In [ ]:
def retornar_horario(data:dict|str, dia:str) -> str:
  """
  retornar_horario: Recibe el diccionario con los horario y el dia en question y
                    devuelve el horario de ese dia
  Args:
      data (dict | str): Diccionario de Horarios | string en caso "No Disponible"
      dia (str): Dia del que se quiere el horario

  Returns:
      str: Horario del dia preguntado
  """
  if data == 'No Disponible':
    return 'No Disponible'
  else:
    if dia in data.keys():
      return data[dia]
    else:
      return "No Disponible"

Creando las Columnas de los distintos dias

In [ ]:
business['monday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Monday'))

business['tuesday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Tuesday'))

business['wednesday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Wednesday'))

business['thursday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Thursday'))

business['friday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Friday'))

business['saturday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Saturday'))

business['sunday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Sunday'))

business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,hours,monday,tuesday,wednesday,thursday,friday,saturday,sunday
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,No Disponible
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
10,UJsufbvfyfONHeWdvAHKjA,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,FL,34639,28.190459,-82.45738,3.5,6,1,"{'Monday': '9:30-21:30', 'Tuesday': '9:30-21:3...",9:30-21:30,9:30-21:30,9:30-21:30,9:30-21:30,9:30-21:30,9:30-21:30,10:0-20:0
37,mGFsPRHdE6SMV1gYQ6GzgQ,T-Mobile,4735 Concord Pike,Wilmington,DE,19803,39.822176,-75.546007,2.0,6,1,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-20:0', ...",0:0-0:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,11:0-18:0
44,mglME3E7esPSpIF5bias8w,Pier 1,1014 Baltimore Pike,Springfield,PA,19064,39.91527,-75.342681,2.5,8,0,"{'Monday': '10:0-20:0', 'Tuesday': '10:0-20:0'...",10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-21:0,10:0-21:0,11:0-19:0


Ahora de se debe borrar la columna `hours` y ya estaría lista: la tabla `y_business`

In [ ]:
business = business.drop(columns=['hours'])

## Ahora crearemos un Dataframe con el archivo `tip.json` tal como viene del exterior

In [ ]:
tip = pd.read_json('datasets/yelp/archivos_originales/tip.json', lines=True)

In [ ]:
tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


Filtraremos los business_id tomando en cuenta la lista que obtuvimos del Maestro de Categorias del Inicio de este cuaderno Jupyter

In [ ]:
mask = tip['business_id'].isin(lista_business_alimentos_bebidas)
tip = tip.loc[mask]

Observando Valores Nulos

In [ ]:
tip.isna().sum()

,0
user_id,0
business_id,0
text,0
date,0
compliment_count,0


Obtendremos solo la fecha

In [ ]:
# Creamos la nueva Columna
tip['date_only'] = tip['date'].dt.date

# Borramos la antigua columna
tip = tip.drop(columns=['date'])

# Renombramos
tip = tip.rename(columns={'date_only': 'date'})

tip.head()

user_id,business_id,text,compliment_count,date
str,str,str,i64,date
"""-copOvldyKh1qr-vzkDEvw""","""MYoRNLb5chwjQe3c_k37Gg""","""It's open even when you think …",0,2013-08-18
"""4ay-fdVks5WMerYL_htkGQ""","""pICJRcyqW1cF96Q3XhLSbw""","""Starbucks substitute in boring…",0,2012-06-09
"""JsXhBw6MntzTJjH_Ue5udw""","""wLHodvVFLTgK3nl2XG8OCw""","""Very good will definitely be c…",0,2017-03-23
"""Ll5l4WTKPH7zWQWA68Rhdw""","""LJaR65ALpz261_dlVWqV2g""","""Helping Mona find a new phone.""",0,2012-06-02
"""phKNHTaokisQkZ7qQbxryA""","""pWuoHHu0LuVrD0HjadDygg""","""Crepes please!!!""",0,2010-11-21


## Ahora crearemos un Dataframe con el archivo `checkin.json` tal como viene del exterior

In [ ]:
checkin = pd.read_json('datasets/yelp/archivos_originales/checkin.json', lines=True)

In [ ]:
checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


Verificación de Nulos

In [ ]:
checkin.isna().sum()

,0
business_id,0
date,0


Crearemos una lista de Fechas a partir de la columna `date`

In [ ]:
checkin['lista_date'] = checkin['date'].str.split(',')
checkin.head()

,business_id,date,lista_date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020...","[2020-03-13 21:10:56, 2020-06-02 22:18:06, 2..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011...","[2010-09-13 21:43:09, 2011-05-04 23:08:15, 2..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22","[2013-06-14 23:29:17, 2014-08-13 23:20:22]"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012...","[2011-02-15 17:12:00, 2011-07-28 02:46:10, 2..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014...","[2014-04-21 20:42:11, 2014-04-28 21:04:46, 2..."


Eliminamos la columna `date`, hacemos explode basandonos en la columna `lista_date`, renombramos la columna y ya está lista la tabla

In [ ]:
checkin = checkin.drop(columns=['date'])
checkin = checkin.explode('lista_date')
checkin = checkin.rename(columns={'lista_date': 'date'})
checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13 21:10:56
0,---kPU91CF4Lq2-WlRu9Lw,2020-06-02 22:18:06
0,---kPU91CF4Lq2-WlRu9Lw,2020-07-24 22:42:27
0,---kPU91CF4Lq2-WlRu9Lw,2020-10-24 21:36:13
0,---kPU91CF4Lq2-WlRu9Lw,2020-12-09 21:23:33


Ahora Vamos a pasar el campo `date` a tipo `Date` y conteniendo solo la fecha

In [ ]:
def devuelve_solo_fecha(campo:str) -> str:
  """
  devuelve_solo_fecha: Devuelve los 10 primeros caracteres de una cadena  
  Args:
      campo (str): cadena recibida

  Returns:
      str: 10 primeros caracteres de la cadena
  """
  return campo.strip()[:10]

# Creo campo solo con la fecha
checkin['only_date'] = checkin['date'].apply(devuelve_solo_fecha)

# Casteo el campo de Fecha
checkin['only_date'] = pd.to_datetime(checkin['only_date'])

# Elimino el antiguo campo fecha
checkin = checkin.drop(columns=['date'])

# renombro columna
checkin = checkin.rename(columns={'only_date': 'date'})

checkin.head()

business_id,date
str,date
"""---kPU91CF4Lq2-WlRu9Lw""",2020-03-13
"""---kPU91CF4Lq2-WlRu9Lw""",2020-06-02
"""---kPU91CF4Lq2-WlRu9Lw""",2020-07-24
"""---kPU91CF4Lq2-WlRu9Lw""",2020-10-24
"""---kPU91CF4Lq2-WlRu9Lw""",2020-12-09


Se procede a Guardar las tablas `y_business`, `y_attributes_business`, `y_checkin` y `y_tip` como van a datawarehouse y ya filtradas

In [ ]:
# Guardando la tabla `y_business`
business.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_business.parquet', index=False)

# Guardando la tabla `y_attributes_business`
attributes.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_attributes_business.parquet', index=False)

# Guardando la tabla `y_checkin`
checkin.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_checkin.parquet', index=False)

# Guardando la tabla `y_tip`
tip.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_tip.parquet', index=False)

Vamos a Borrar Variables para Liberar Memoria y poder seguir Trabajando

In [ ]:
del business

del attributes

del checkin

del tip

gc.collect()

76

## Vamos a cargar el archivo `review.json` tal como viene original del exterior

In [ ]:
reviews = pd.read_ndjson("datasets/yelp/review.json", lines=True)

In [ ]:
reviews.head()

review_id,user_id,business_id,stars,useful,funny,cool,text,date
str,str,str,f64,i64,i64,i64,str,str
"""KU_O5udG6zpxOg-VcAEodg""","""mh_-eMZ6K5RLWhZyISBhwA""","""XQfwVwDr-v0ZS3_CbbE5Xw""",3.0,0,0,0,"""If you decide to eat here, jus…","""2018-07-07 22:09:11"""
"""BiTunyQ73aT9WBnpR9DZGw""","""OyoGAe7OKpv6SyGZT5g77Q""","""7ATYjTIgM3jUlt4UM3IypQ""",5.0,1,0,1,"""I've taken a lot of spin class…","""2012-01-03 15:28:18"""
"""saUsX_uimxRlCVr67Z4Jig""","""8g_iMtfSiwikVnbP2etR0A""","""YjUWPpI6HXG530lwP-fb2A""",3.0,0,0,0,"""Family diner. Had the buffet. …","""2014-02-05 20:30:30"""
"""AqPFMleE6RsU23_auESxiA""","""_7bHUi9Uuf5__HHc_Q8guQ""","""kxX2SOes4o-D3ZQBkiMRfA""",5.0,1,0,1,"""Wow! Yummy, different, delic…","""2015-01-04 00:01:03"""
"""Sx8TMOWLNuJBWer-0pcmoA""","""bcjbaE6dDog4jkNY91ncLQ""","""e4Vwtrqf-wpJfwesgvdgxQ""",4.0,1,0,1,"""Cute interior and owner (?) ga…","""2017-01-14 20:54:15"""


## Filtremos las reviews de los bussines de la categoria alimentos y bebidas

In [ ]:
mask = reviews['business_id'].isin(lista_business_alimentos_bebidas)

reviews = reviews.loc[mask]

Revisando cuantos Nulos vienen

In [ ]:
for column in reviews.columns:
    print(f'Columna  {column} -> {reviews[column].isnull().sum()} ')

Columna  review_id -> 0 
Columna  user_id -> 0 
Columna  business_id -> 0 
Columna  stars -> 0 
Columna  useful -> 0 
Columna  funny -> 0 
Columna  cool -> 0 
Columna  text -> 0 
Columna  date -> 0 


In [ ]:
print(f'Cantidad de Filas -> {reviews.shape[0]}')
print(f'Cantidad reviews_id  No repetidas -> {reviews["review_id"].unique().shape[0]}')
print(f'Cantidad filas No repetidas -> {reviews.unique().shape[0]}')

Cantidad de Filas -> 1297614
Cantidad reviews_id  No repetidas -> 1297614
Cantidad filas No repetidas -> 1297614


In [ ]:
reviews.head(1)

review_id,user_id,business_id,stars,useful,funny,cool,text,date
str,str,str,f64,i64,i64,i64,str,str
"""AqPFMleE6RsU23_auESxiA""","""_7bHUi9Uuf5__HHc_Q8guQ""","""kxX2SOes4o-D3ZQBkiMRfA""",5.0,1,0,1,"""Wow! Yummy, different, delic…","""2015-01-04 00:01:03"""


Vamos a crear una columna que sea solo fecha

In [ ]:
# Creo campo solo con la fecha
reviews['only_date'] = reviews['date'].apply(devuelve_solo_fecha)

# Casteo el campo de Fecha
reviews['only_date'] = pd.to_datetime(reviews['only_date'])

# Elimino el antiguo campo fecha
reviews = reviews.drop(columns=['date'])

# renombro columna
reviews = reviews.rename(columns={'only_date': 'date'})

reviews.head()

review_id,user_id,business_id,stars,useful,funny,cool,text,date
str,str,str,f64,i64,i64,i64,str,date
"""AqPFMleE6RsU23_auESxiA""","""_7bHUi9Uuf5__HHc_Q8guQ""","""kxX2SOes4o-D3ZQBkiMRfA""",5.0,1,0,1,"""Wow! Yummy, different, delic…",2015-01-04
"""Sx8TMOWLNuJBWer-0pcmoA""","""bcjbaE6dDog4jkNY91ncLQ""","""e4Vwtrqf-wpJfwesgvdgxQ""",4.0,1,0,1,"""Cute interior and owner (?) ga…",2017-01-14
"""l3Wk_mvAog6XANIuGQ9C7Q""","""ZbqSHbgCjzVAqaa7NKWn5A""","""EQ-TZ2eeD_E0BHuvoaeG5Q""",4.0,0,0,0,"""Locals recommended Milktooth, …",2015-08-19
"""lUUhg8ltDsUZ9h0xnwY4Dg""","""RreNy--tOmXMl1en0wiBOg""","""cPepkJeRMtHapc_b2Oe_dw""",4.0,1,0,1,"""I was really between 3 and 4 s…",2018-07-17
"""xdQe2E8jR6pLA07p1aYtKA""","""yFuE8SPF-d1GXJUWYgKtzg""","""X8lfGPagfLzfOsx0k08NRQ""",5.0,0,0,0,"""Took my vehicle here for some …",2017-05-13


Escogiendo las reseñas a partir del 2021

In [ ]:
print(f'Cantidad de Filas -> {reviews.shape[0]}')

Cantidad de Filas -> 1297614


In [ ]:
mask = reviews['date'] > datetime.strptime('2020-12-31', '%Y-%m-%d')
reviews = reviews.loc[mask]

print(f'Cantidad de Filas a partir del 2021 -> {reviews.shape[0]}')

Cantidad de Filas a partir del 2021 -> 115290


Toca Guardar la Tabla `y_reviews`

In [ ]:
# Guardando la tabla `y_reviews`
reviews.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_review.parquet', index=False)

Antes de Borrar la variable `reviews` necesito obtener la lista de los id de los usuarios para poder filtrar la información de usuarios

In [ ]:
lista_usuarios_a_filtrar = reviews['user_id'].unique().tolist()

Ahora si puedo Borrar la variable y liberar memoria para seguir trabajando

In [ ]:
del reviews
gc.collect()

204

## Vamos a cargar el archivo `user.parquet` tal como viene original del exterior

In [ ]:
user = pd.read_parquet("datasets/yelp/archivos_originales/user.parquet")

In [ ]:
user.head()

user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
str,str,i64,str,i64,i64,i64,str,str,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""qVc8ODYU5SZjKXVBgXdI7w""","""Walker""",585,"""2007-01-25 16:47:26""",7217,1259,5994,"""2007""","""NSCy54eWehBJyZdG2iE84w, pe42u7…",267,3.91,250,65,55,56,18,232,844,467,467,239,180
"""j14WgRoU_-2ZE1aw1dXrJg""","""Daniel""",4333,"""2009-01-25 04:35:42""",43091,13066,27281,"""2009,2010,2011,2012,2013,2014,…","""ueRPE0CX75ePGMqOFVj6IQ, 52oH4D…",3138,3.74,1145,264,184,157,251,1847,7054,3131,3131,1521,1946
"""0G-QF457q_0Z_jKqh6xWiA""","""Joi""",2009,"""2008-03-12 19:13:45""",27137,14049,20385,"""2009,2010,2011,2012,2013,2014,…","""K8h4ypoVZAnmhC2dHKsDrQ, 7nS_R5…",944,4.1,1729,129,105,152,76,2073,6409,2442,2442,1117,462
"""ouODopBKF3AqfCkuQEnrDg""","""Danan""",2108,"""2010-04-09 05:28:01""",4527,1110,2850,"""2010,2011,2012,2013,2014,2015,…","""tlEHfYMPIm8-AMMk3T1s0w, aOkQtJ…",134,3.57,62,15,20,10,1,65,87,89,89,92,6
"""1ziEzn7BoXTfOQ4YBkdz1Q""","""Christine""",703,"""2008-05-07 20:40:15""",1577,430,699,"""2009,2010,2011,2012,2013,2014,…","""U0JD-MOcBHmoRLYaaB7m1g, elIIKD…",53,3.81,16,5,4,4,0,40,57,63,63,31,8


## Debemos filtrar según la lista, `lista_usuarios_a_filtrar`

In [ ]:
mask = user['user_id'].is_in(lista_usuarios_a_filtrar)
user = user.loc[mask]

Verificando los Nulos

In [ ]:
for column in user.columns:
    print(f"Columna {column}: {user[column].isnull().sum()}")

Columna user_id: 0
Columna name: 0
Columna review_count: 0
Columna yelping_since: 0
Columna useful: 0
Columna funny: 0
Columna cool: 0
Columna elite: 0
Columna friends: 0
Columna fans: 0
Columna average_stars: 0
Columna compliment_hot: 0
Columna compliment_more: 0
Columna compliment_profile: 0
Columna compliment_cute: 0
Columna compliment_list: 0
Columna compliment_note: 0
Columna compliment_plain: 0
Columna compliment_cool: 0
Columna compliment_funny: 0
Columna compliment_writer: 0
Columna compliment_photos: 0


Comprobando si hay filas repetidas

In [ ]:
print(f"Total Filas como viene la data {user.shape[0]}")

print(f"Total filas Unicas {user.unique().shape[0]}")

Total Filas como viene la data 91630
Total filas Unicas 85958


In [ ]:
user = user.unique()

user.shape[0]

85958

Del campo `yelping_since` que es de tipo String saquemos uno tipo Date

In [ ]:
# Creo campo solo con la fecha
user['date'] = reviews['yelping_since'].apply(devuelve_solo_fecha)

# Casteo el campo de Fecha
user['only_date'] = pd.to_datetime(user['date'])

# Elimino el antiguo campo fecha
user = user.drop(columns=['yelping_since'])

# renombro columna
user = user.rename(columns={'date': 'yelping_since'})

Ahora proseguimos creando la Tabla `y_friends_user`

In [ ]:
friends_user = user[['user_id', 'friends']]

friends_user.columns = ['user_id', 'friend_id']

In [ ]:
friends_user.head()

user_id,friend_id
str,str
"""yKofALcKQfWBEpNbDLXdWw""","""b_hOVL43ocK8Shn91hdlHQ, PoDa9_…"
"""tUOD2bXFLaiYP5_KeKJNOA""","""mHsYHxQKWfF7LvRCuJxQtg, gwzCrN…"
"""KvmbHF6a4lQr-yRTQWoWaA""","""mgUlio5hcNA2rjODrkuGww, vt5skq…"
"""_cgVicVyTX6OXV2sK-ULzw""","""None"""
"""2xcWWr34eFP8dDIPot2CLA""","""gxJlcUCUO8Xk1bDOqcYnBg, kD3ZSt…"


Creando la Tabla `y_elite_user`

In [ ]:
elite_user = user[['user_id', 'elite']]

In [ ]:
elite_user.head()

user_id,elite
str,str
"""yKofALcKQfWBEpNbDLXdWw""","""2018,2019,20,20,2021"""
"""tUOD2bXFLaiYP5_KeKJNOA""","""2015,2016,2018,2019"""
"""KvmbHF6a4lQr-yRTQWoWaA""",""""""
"""_cgVicVyTX6OXV2sK-ULzw""",""""""
"""2xcWWr34eFP8dDIPot2CLA""","""2016,2017,2019,20,20,2021"""


Ahora podemos Borrar las columnas `friends` y `elite` de la tabla `yelp_user`, luego vamos a guardar esta tabla y por último borraremos la variable para liberar memoria

In [ ]:
# Elimino los campos
user = user.drop(columns=['friends', 'elite'])

user.head()

user_id,name,review_count,useful,funny,cool,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,yelping_since
str,str,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,date
"""yKofALcKQfWBEpNbDLXdWw""","""Don""",276,642,96,258,26,3.73,2,2,0,0,0,9,16,15,15,3,4,2011-11-08
"""tUOD2bXFLaiYP5_KeKJNOA""","""Jeramy""",175,208,43,80,16,4.26,3,1,0,0,0,6,6,7,7,8,1,2011-07-29
"""KvmbHF6a4lQr-yRTQWoWaA""","""Robin""",1,0,0,0,0,5.0,0,0,0,0,0,0,0,0,0,0,0,2018-07-08
"""_cgVicVyTX6OXV2sK-ULzw""","""Ronald""",1,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,2019-12-27
"""2xcWWr34eFP8dDIPot2CLA""","""Shawna""",134,169,41,111,6,4.77,1,1,1,2,0,9,7,5,5,5,4,2013-06-16


In [ ]:
user.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_user.parquet', index=False)

In [ ]:
# BORRANDO VARIABLE Y LIBERANDO MEMORIA
del user
gc.collect()

164

Toca trabajar la tabla `y_friends_user`

In [ ]:
print(friends_user.shape)
friends_user.head()

(85958, 2)


user_id,friend_id
str,str
"""yKofALcKQfWBEpNbDLXdWw""","""b_hOVL43ocK8Shn91hdlHQ, PoDa9_…"
"""tUOD2bXFLaiYP5_KeKJNOA""","""mHsYHxQKWfF7LvRCuJxQtg, gwzCrN…"
"""KvmbHF6a4lQr-yRTQWoWaA""","""mgUlio5hcNA2rjODrkuGww, vt5skq…"
"""_cgVicVyTX6OXV2sK-ULzw""","""None"""
"""2xcWWr34eFP8dDIPot2CLA""","""gxJlcUCUO8Xk1bDOqcYnBg, kD3ZSt…"


Eliminando los 'None'

In [ ]:
mask = friends_user['friend_id'] == 'None'

friends_user = friends_user.loc[~mask]

print(friends_user.shape)
friends_user.head()

(46522, 2)


user_id,friend_id
str,str
"""yKofALcKQfWBEpNbDLXdWw""","""b_hOVL43ocK8Shn91hdlHQ, PoDa9_…"
"""tUOD2bXFLaiYP5_KeKJNOA""","""mHsYHxQKWfF7LvRCuJxQtg, gwzCrN…"
"""KvmbHF6a4lQr-yRTQWoWaA""","""mgUlio5hcNA2rjODrkuGww, vt5skq…"
"""2xcWWr34eFP8dDIPot2CLA""","""gxJlcUCUO8Xk1bDOqcYnBg, kD3ZSt…"
"""jiRfSBKVWZ0cyAbk5Rvhww""","""w6SWiln7NtQYoM4RcHaqaA, xZnpcE…"


Se crea una columna nueva que sea de tipo LISTA a partir de `friend_id` y luego debemos borrar esta ultima

In [ ]:
friends_user['lista_friends'] = friends_user['friend_id'].apply(lambda x: x.split(','))

In [ ]:
friends_user = friends_user.drop(columns=['friend_id'])

print(friends_user.shape)
friends_user.head()

(46522, 2)


user_id,lista_friends
str,list[str]
"""yKofALcKQfWBEpNbDLXdWw""","[""b_hOVL43ocK8Shn91hdlHQ"", "" PoDa9_c1Z0nk2dYtsWcKCQ"", … "" wyym9Y1SLEF4oz7mz4J5Xw""]"
"""tUOD2bXFLaiYP5_KeKJNOA""","[""mHsYHxQKWfF7LvRCuJxQtg"", "" gwzCrNAD7MGMZYu7MtEh0w"", … "" pGANa5aQeF2YpGk5CLhd1A""]"
"""KvmbHF6a4lQr-yRTQWoWaA""","[""mgUlio5hcNA2rjODrkuGww"", "" vt5skqY3woHvFsQuqTsWVQ"", … "" L6scs33tzFNCsjVMbJ3iXg""]"
"""2xcWWr34eFP8dDIPot2CLA""","[""gxJlcUCUO8Xk1bDOqcYnBg"", "" kD3ZSt0_yWUjl3HStgVcOA"", … "" fCJmfg8MTgN_d_LeU7vQ6Q""]"
"""jiRfSBKVWZ0cyAbk5Rvhww""","[""w6SWiln7NtQYoM4RcHaqaA"", "" xZnpcE5rVlufSYuBzDpA0g"", … "" -4AMxFUpXWYjekSdrg3XUA""]"


Se utiliza el metodo `explode` de los dataframes y luego se cambian los nombres de las columnas

In [ ]:
friends_user = friends_user.explode('lista_friends')

friends_user.columns = ['user_id', 'friend_id']

print(friends_user.shape)
friends_user.head()

(5450645, 2)


user_id,friend_id
str,str
"""yKofALcKQfWBEpNbDLXdWw""","""b_hOVL43ocK8Shn91hdlHQ"""
"""yKofALcKQfWBEpNbDLXdWw""",""" PoDa9_c1Z0nk2dYtsWcKCQ"""
"""yKofALcKQfWBEpNbDLXdWw""",""" XeaqHoiS4M8PyYN4Ubyrcw"""
"""yKofALcKQfWBEpNbDLXdWw""",""" io7zHoTohIjj_SO4mvlVug"""
"""yKofALcKQfWBEpNbDLXdWw""",""" LwS4Vqoy-rK6vbHyU_5vaQ"""


Guardando la Tabla `y_friends_user`

In [ ]:
friends_user.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_friends_user.parquet', index=False)

Borraremos la Variable `friends_user` y liberaremos memoria

In [ ]:
del friends_user
gc.collect()

170

Ahora toca trabajar con la tabla `y_elite_user`

In [ ]:
elite_user.head()

user_id,elite
str,str
"""yKofALcKQfWBEpNbDLXdWw""","""2018,2019,20,20,2021"""
"""tUOD2bXFLaiYP5_KeKJNOA""","""2015,2016,2018,2019"""
"""KvmbHF6a4lQr-yRTQWoWaA""",""""""
"""_cgVicVyTX6OXV2sK-ULzw""",""""""
"""2xcWWr34eFP8dDIPot2CLA""","""2016,2017,2019,20,20,2021"""


In [ ]:
mask = elite_user['elite'] == ''

print(f'Total filas --> {elite_user.shape[0]}')

print(f'Total filas con informacion --> {elite_user.loc[~mask].shape[0]}')

print(f'Total filas SIN informacion --> {elite_user.loc[mask].shape[0]}')

Total filas --> 85958
Total filas CON informacion --> 9377
Total filas SIN informacion --> 76581


#### Se procederá con el proceso para obtener la tabla en el datawarehouse, Quedando a consideración sí se utiliza o no esta data para entrenar el modelo...

Toca Borrar las filas sin información

In [ ]:
mask = elite_user['elite'] == ''

elite_user = elite_user.loc[~mask]

print(elite_user.shape)
elite_user.head()

(9377, 2)


user_id,elite
str,str
"""yKofALcKQfWBEpNbDLXdWw""","""2018,2019,20,20,2021"""
"""tUOD2bXFLaiYP5_KeKJNOA""","""2015,2016,2018,2019"""
"""2xcWWr34eFP8dDIPot2CLA""","""2016,2017,2019,20,20,2021"""
"""jiRfSBKVWZ0cyAbk5Rvhww""","""2018,2019,20,20,2021"""
"""hqcqfT_h59tU6mcLW2XnaA""","""2021"""


Toca crear una columna nueva que sea de tipo Lista a partir del String que hay en la columna `elite`

In [ ]:
elite_user['year'] = elite_user['elite'].apply(lambda x: x.split(','))
elite_user.head()

user_id,elite,year
str,str,list[str]
"""yKofALcKQfWBEpNbDLXdWw""","""2018,2019,20,20,2021""","[""2018"", ""2019"", … ""2021""]"
"""tUOD2bXFLaiYP5_KeKJNOA""","""2015,2016,2018,2019""","[""2015"", ""2016"", … ""2019""]"
"""2xcWWr34eFP8dDIPot2CLA""","""2016,2017,2019,20,20,2021""","[""2016"", ""2017"", … ""2021""]"
"""jiRfSBKVWZ0cyAbk5Rvhww""","""2018,2019,20,20,2021""","[""2018"", ""2019"", … ""2021""]"
"""hqcqfT_h59tU6mcLW2XnaA""","""2021""","[""2021""]"


Borrando la columna `elite`

In [ ]:
elite_user = elite_user.drop(columns=['elite'])

Aplicando el metodo `explode` a la columna `year` que hasta ahora es de tipo Lista

In [ ]:
elite_user = elite_user.explode('year')

elite_user.head()

user_id,year
str,str
"""yKofALcKQfWBEpNbDLXdWw""","""2018"""
"""yKofALcKQfWBEpNbDLXdWw""","""2019"""
"""yKofALcKQfWBEpNbDLXdWw""","""20"""
"""yKofALcKQfWBEpNbDLXdWw""","""20"""
"""yKofALcKQfWBEpNbDLXdWw""","""2021"""


Eliminando repetidos y Sustituyendo los 20 por 2020

In [ ]:
elite_user = elite_user.unique()

elite_user.loc[elite_user['year'] == '20', 'year'] = '2020'

elite_user.head()

user_id,year
str,str
"""T4L1mTz2Shd9VpVIb1fZoQ""","""2018"""
"""5fBL_DYLnphhCcfSeomyhg""","""2019"""
"""_uMEeZbgqigt222bZ642qQ""","""2019"""
"""RQWQTxfhGGJ7bGgJ1EnExQ""","""2017"""
"""yhJ1SVBSjC_R2viEh5AbUA""","""2017"""


Ahora debemos Castear a Entero la columna `year`

In [ ]:
elite_user['year'] = elite_user['year'].astype(int)

elite_user.head()

user_id,year
str,i64
"""T4L1mTz2Shd9VpVIb1fZoQ""",2018
"""5fBL_DYLnphhCcfSeomyhg""",2019
"""_uMEeZbgqigt222bZ642qQ""",2019
"""RQWQTxfhGGJ7bGgJ1EnExQ""",2017
"""yhJ1SVBSjC_R2viEh5AbUA""",2017


Guardando la Tabla `y_elite_user`

In [ ]:
elite_user.to_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_elite_user.parquet', index=False)